In [1]:
import json
import pandas as pd
from pathlib import Path
from datasets import load_dataset

In [2]:
ds = load_dataset("hal-utokyo/MangaVQA", cache_dir="/Utilisateurs/umushtaq/emorec_work/mdlt_er/datasets/mangavqa")

README.md:   0%|          | 0.00/874 [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/39.0k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/526 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    test: Dataset({
        features: ['title', 'image_number', 'question', 'answer', 'required_info', 'w5h1', 'answer_type'],
        num_rows: 526
    })
})

In [4]:
ds['test'][0]

{'title': 'LoveHina_vol14',
 'image_number': 2,
 'question': '夕方のシーンで、2人がどこに行くことを決めたか？',
 'answer': 'ひなた荘',
 'required_info': 'panel',
 'w5h1': 'where',
 'answer_type': 'exact_extraction'}

In [5]:
df = pd.DataFrame(ds['test']) 
df.head()

,title,image_number,question,answer,required_info,w5h1,answer_type
0,LoveHina_vol14,2,夕方のシーンで、2人がどこに行くことを決めたか？,ひなた荘,panel,where,exact_extraction
1,LoveHina_vol14,3,瀬田くんはどうしてここに残りたいと言っていますか？,大きな遺跡を発掘しながらそこの土地で生活するのが夢だから,panel,why,descriptive_answering
2,LoveHina_vol14,3,はるかさんはお店についてどう言及していますか？,つぐなり潰すなり好きにしてくれと言っています。,panel,what,descriptive_answering
3,LoveHina_vol14,5,左下に場面で、男性が赤面しているのはどうしてでしょうか？,Faxで提示された条件の中に、帰った時に景太郎の隣にいる女性を婚約者とみなすと書かれていたから。,panel,why,descriptive_answering
4,LoveHina_vol14,7,部屋に到着時、登場人物たちが不思議がっていたのはどうしてですか？,おばあちゃんがいなかったから,page,why,descriptive_answering


In [6]:
def generation_instruction(row):
    

   instruction = f"""あなたは視覚的質問応答タスクの専門家です。画像を分析し、その内容に関する質問に答えてください。

画像を注意深く観察し、質問に対して正確かつ簡潔に回答してください。

回答はJSON形式で返してください:
{{"answer": "<回答>"}}

説明やコメントは不要です。JSONオブジェクトのみを返してください。

   """

   return instruction

In [8]:
def build_output(row):
        
    return {"answer": row.answer}

In [9]:
df["instruction"] = df.apply(lambda x: generation_instruction(x), axis=1)

In [10]:
df["answer"] = df.apply(lambda x: build_output(x), axis=1) # type: ignore

In [11]:
print(df.iloc[0]['instruction'])

あなたは視覚的質問応答タスクの専門家です。画像を分析し、その内容に関する質問に答えてください。

画像を注意深く観察し、質問に対して正確かつ簡潔に回答してください。

回答はJSON形式で返してください:
{"answer": "<回答>"}

説明やコメントは不要です。JSONオブジェクトのみを返してください。

   


In [12]:
print(df.iloc[0]['answer'])

{'answer': 'ひなた荘'}


In [13]:
def build_jsonl_dataset(
    split,
    image_col="image_number",
    instruction_col="instruction",
    answer_col="answer",
    
):  
    output_path=f"mangavqa_{split}.jsonl"
    records = []
    
    #df = df_train if split == "train" else df_test

    for _, row in df.iterrows():
        sample = {
            "messages": [
            {
                "content": "<image>" + row[instruction_col],
                "role": "user"
            },
            {
                "content": "" + str(row[answer_col]),
                "role": "assistant"
            },
            ],
            "images": [
            row[image_col]
            ]
        }
        records.append(sample)
        
    output_path = Path(output_path)
    with output_path.open("w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    print(f"Saved {len(records)} samples to {output_path}")

In [14]:
build_jsonl_dataset(split="train")

Saved 526 samples to mangavqa_train.jsonl
